<a href="https://colab.research.google.com/github/Dywinar/project_clasifaier/blob/main/Project_KL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Загрузка данных, подсчет пропусков

In [91]:
df = pd.read_csv('/content/users_behavior.csv')
df.drop('Unnamed: 0', axis= 1, inplace= True )
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Считаем пропуски

In [95]:
df.isnull().sum() # их нет

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

 Поскольку для лучшего обучения, нужно использовать нормализаци, тогда:

In [99]:
X = np.array(df[df.columns[:-1]])
y = df['is_ultra']
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Делаем обьект класса для подсчета наилучшей модели

Строем обьект класса

In [103]:
class best_model():
  def __init__(self, model):
    self.model = model
    self.accuracy = None
    self.f_1 = None
    self.best_models = None
  def fit(self, X_train,X_test,  y_train, y_test, parametrs):
    """
    Находит наилучшую модель, затем обучает
    """
    self.best_models = GridSearchCV(param_grid=parametrs,estimator=self.model)
    self.best_models.fit(X_train, y_train)
    return self.all_metrics([X_test, y_test])
  def predict(self, X) -> np.ndarray:
    """
    Предсказание модели
    """
    return self.best_models.predict(X)
  def all_metrics(self,mas):
    """
    Подсчет  необходимых метрик
    """
    print(f"Accuracy : ", accuracy_score(self.predict(mas[0]), mas[1]), f"F1 score: ", f1_score(self.predict(mas[0]),mas[1]))
    return accuracy_score(self.predict(mas[0]), mas[1])

# Приступаем к поиску

In [105]:
models = [[LogisticRegression(), {"penalty" : ['l2', None], "solver": ['sag', 'saga'], 'max_iter': [100, 1000, 1200]}],
          [KNeighborsClassifier(), {'n_neighbors' : [2, 5, 7, 8, 10]}],
          [tree.DecisionTreeClassifier(), {'min_samples_split': list(range(2,11)),'max_depth' : [1,2,3,4],'criterion' :['gini', 'entropy']}],
          [RandomForestClassifier(),{'n_estimators' : [30, 100, 300, 10],'max_depth' : [10, 20, 30],'min_samples_split': [2,5,10],'min_samples_leaf' : [1,2,4],'criterion': ['gini', 'entropy']} ]
          ]
max_accuracy = {}
for model in models:
   best_ml = best_model(model[0])
   max_accuracy[str(model[0])] = best_ml.fit(X_train,X_test,  y_train, y_test, model[1])
   print("Закончен:", str(model[0]))

Accuracy :  0.7651632970451011 F1 score:  0.41245136186770426
Закончен: LogisticRegression()
Accuracy :  0.8180404354587869 F1 score:  0.6086956521739131
Закончен: KNeighborsClassifier()
Accuracy :  0.7962674961119751 F1 score:  0.576051779935275
Закончен: DecisionTreeClassifier()
Accuracy :  0.8242612752721618 F1 score:  0.64576802507837
Закончен: RandomForestClassifier()


Наилучшая модель тогда:

In [123]:
max_dict = [(x, y) for x,y in max_accuracy.items()]
print(*max(max_dict, key=lambda x: x[1]))

RandomForestClassifier() 0.8242612752721618
